In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from nn_robust_attacks.setup_mnist import MNIST, MNISTModel

Using TensorFlow backend.


In [2]:
sess = tf.Session(config=tf.ConfigProto())
K.set_session(sess)

data = MNIST()

# CNN Model for MNIST 
class MNIST_Model:
    def __init__(self, session=None):
        self.num_channels = 1
        self.image_size = 28
        self.num_labels = 10

        model = Sequential()

        model.add(Conv2D(32, (3, 3),
                         input_shape=(28, 28, 1)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Flatten())
        model.add(Dense(200))
        model.add(Activation('relu'))
        model.add(Dense(200))
        model.add(Activation('relu'))
        model.add(Dense(10))

        self.model = model

    def predict(self, data):
        return self.model(data)

# Model Training

In [3]:
from keras.models import load_model, Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD

def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
                                                   logits=predicted/1)
modelname = "models/trained_model1"
model1 = MNIST_Model()
model1.model = load_model(modelname, custom_objects={'fn':fn})
        
modelname = "models/trained_model2"
model2 = MNIST_Model()
model2.model = load_model(modelname, custom_objects={'fn':fn})

modelname = "models/trained_model3"
model3 = MNIST_Model()
model3.model = load_model(modelname, custom_objects={'fn':fn})

modelname = "models/trained_model4"
model4 = MNIST_Model()
model4.model = load_model(modelname, custom_objects={'fn':fn})

modelname = "models/trained_model5"
model5 = MNIST_Model()
model5.model = load_model(modelname, custom_objects={'fn':fn})


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.




# Preprocess / Setup

In [4]:
from keras.datasets import mnist as data_keras
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = data_keras.load_data()
x_train = x_train[...,np.newaxis] /255.0
x_test = x_test[...,np.newaxis] / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Helper Functions

In [5]:
from numpy import savetxt

def save_results(inputs, adv, subset_attacked):
    np.save('four_subset_FGSM/inputs_target9_' + '_' + subset_attacked + '.csv', inputs)
    np.save('four_subset_FGSM/adv_target9_' + subset_attacked + '.csv', adv)

In [6]:
def get_label_confidence(image, model):
    image = image[0]
    pr = model.predict(image)
    pr = np.array([np.exp(x) for x in pr])
    pr = pr / np.sum(pr)
    image_class = model.predict_classes(image)
    return image_class[0], max(max(pr))

In [7]:
def attack_evaluation(num_img, adv):
    l = []
    for i in range(0, num_img):
        res = [get_label_confidence(adv[i][np.newaxis, ...], model1.model), get_label_confidence(adv[i][np.newaxis, ...], model2.model), get_label_confidence(adv[i][np.newaxis, ...], model3.model), get_label_confidence(adv[i][np.newaxis, ...], model4.model)]
        print(res)
        l.append(res)
    return l

In [8]:
def size_of_attack(adv, inputs, num_img):
    perturb = adv - inputs
    perturb_mean =  np.sum(np.abs(perturb)) / num_img 
    perturb_std = np.std(np.array([sum(x) for x in perturb]))
    print('Mean', perturb_mean)
    print('Standard Deviation', perturb_std)

In [9]:
def transferability(num_img, adv):
    for i in range(0, num_img):
        print(i)
        print('Model 5', get_label_confidence(adv[i][np.newaxis, ...], model5.model))

In [10]:
def check_correct(subset, num_img, adv, target_label=9, true_label=0):
    l = attack_evaluation(num_img, adv)
    correct = []
    for res in l:
        if subset == '1':
            output = True if res[0][0] == target_label and res[1][0] == true_label and res[2][0] == true_label and res[3][0] == true_label else False
        elif subset == '2':
            output = True if res[0][0] == true_label and res[1][0] == target_label and res[2][0] == true_label and res[3][0] == true_label else False
        elif subset == '3':
            output = True if res[0][0] == true_label and res[1][0] == true_label and res[2][0] == target_label and res[3][0] == true_label else False
        elif subset == '4':
            output = True if res[0][0] == true_label and res[1][0] == true_label and res[2][0] == true_label and res[3][0] == target_label else False
        elif subset == '12':
            output = True if res[0][0] == target_label and res[1][0] == target_label and res[2][0] == true_label and res[3][0] == true_label else False
        elif subset == '13':
            output = True if res[0][0] == target_label and res[1][0] == true_label and res[2][0] == target_label and res[3][0] == true_label else False
        elif subset == '14':
            output = True if res[0][0] == target_label and res[1][0] == true_label and res[2][0] == true_label and res[3][0] == target_label else False
        elif subset == '23':
            output = True if res[0][0] == true_label and res[1][0] == target_label and res[2][0] == target_label and res[3][0] == true_label else False
        elif subset == '24':
            output = True if res[0][0] == true_label and res[1][0] == target_label and res[2][0] == true_label and res[3][0] == target_label else False
        elif subset == '34':
            output = True if res[0][0] == true_label and res[1][0] == true_label and res[2][0] == target_label and res[3][0] == target_label else False
        elif subset == '123':
            output = True if res[0][0] == target_label and res[1][0] == target_label and res[2][0] == target_label and res[3][0] == true_label else False
        elif subset == '124':
            output = True if res[0][0] == target_label and res[1][0] == target_label and res[2][0] == true_label and res[3][0] == target_label else False
        elif subset == '134':
            output = True if res[0][0] == target_label and res[1][0] == true_label and res[2][0] == target_label and res[3][0] == target_label else False
        elif subset == '234':
            output = True if res[0][0] == true_label and res[1][0] == target_label and res[2][0] == target_label and res[3][0] == target_label else False
        elif subset == '1234':
            output = True if res[0][0] == target_label and res[1][0] == target_label and res[2][0] == target_label and res[3][0] == target_label else False
            
        if output:
            correct.append(l.index(res))
    return correct

In [11]:
def print_results(num_img, adv, inputs, subset):
    print(check_correct(subset, num_img, adv))
    print(size_of_attack(adv, inputs, num_img))
    print(transferability(num_img, adv))

In [12]:
# Set variables
def FGSM(img, subset, t1, t2, t3, t4, epochs=31, epsilon=0.075, true_class=0):
    prev_probs = []
    prev_probs2 = []
    prev_probs3 = []
    prev_probs4 = []
    prev_adv = []
    adv = []

    x = img
    x_noise = np.zeros_like(x)
    x_adv = x

    for i in range(epochs): 
        # One hot encode the target class
        target1 = K.one_hot(t1, 10)
        target2 = K.one_hot(t2, 10)
        target3 = K.one_hot(t3, 10)
        target4 = K.one_hot(t4, 10)

        true = K.one_hot(true_class, 10)

        # Get the loss and gradient of the loss wrt the inputs
        loss = -1 * K.categorical_crossentropy(target1, K.softmax(model1.model.output))
        loss2 = -1 * K.categorical_crossentropy(target2, K.softmax(model2.model.output))
        loss3 = -1 * K.categorical_crossentropy(target3, K.softmax(model3.model.output))
        loss4 = -1 * K.categorical_crossentropy(target4, K.softmax(model4.model.output))

        grads = K.gradients(loss, model1.model.input)
        grads2 = K.gradients(loss2, model2.model.input)
        grads3 = K.gradients(loss3, model3.model.input)
        grads4 = K.gradients(loss4, model4.model.input)

        # Get the sign of the gradient
        delta = K.sign(grads[0]) + K.sign(grads2[0]) + K.sign(grads3[0]) + K.sign(grads4[0])
        x_noise = x_noise + delta

        # Perturb the image
        x_adv = x_adv + epsilon * delta

        # Get the new image and predictions
        x_adv = sess.run(x_adv, feed_dict={model1.model.input:x, model2.model.input:x, model3.model.input:x, model4.model.input:x})
        prev_adv.append(x_adv)

        preds = model1.model.predict(x_adv)
        preds2 = model2.model.predict(x_adv)
        preds3 = model3.model.predict(x_adv)
        preds4 = model4.model.predict(x_adv)

        if(t1 == np.argmax(preds[0]) and t2 == np.argmax(preds2[0]) and t3 == np.argmax(preds3[0]) and t4 == np.argmax(preds4[0])):
            return x_adv

        # Store the probability of the target class
        prev_probs.append(preds[0][t1])
        prev_probs2.append(preds2[0][t2])
        prev_probs3.append(preds3[0][t3])
        prev_probs4.append(preds4[0][t4])

        x = x_adv

    return x

In [13]:
t1 = np.array([9] * 25)
t2 = np.array([9] * 25)
t3 = np.array([9] * 25)
t4 = np.array([9] * 25)
true = np.load('four_subset_FGSM/precise_large_true.csv.npy')
#t1 = true
t2 = true
t3 = true
t4 = true

subset = '1'
res = []
num_img = 25

for x in range(0, 25):
    print(x)
    img = np.load('inputs_0.csv.npy')[x:x+1]
    adv = FGSM(img, subset, t1[x], t2[x], t3[x], t4[x])
    res.append(adv)

res = np.array(res)
np.save('four_subset_FGSM/adv_large_' + subset + '.csv', res)
print_results(num_img, res, img, subset)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
[(9, 0.59186184), (0, 1.0), (0, 1.0), (0, 1.0)]
[(9, 0.6140026), (0, 1.0), (0, 1.0), (0, 0.9999997)]
[(9, 0.2880941), (0, 0.99999815), (0, 0.99999726), (0, 0.99999946)]
[(9, 0.27036476), (0, 1.0), (0, 1.0), (0, 0.99999994)]
[(9, 0.6265406), (0, 1.0), (0, 0.999997), (0, 0.9999998)]
[(9, 0.9486661), (0, 0.9999992), (0, 0.99999744), (0, 0.99994904)]
[(9, 0.9839688), (0, 1.0), (0, 1.0), (0, 1.0)]
[(9, 0.8039062), (0, 0.9999997), (0, 0.9999804), (0, 0.999999)]
[(9, 0.74934226), (0, 1.0), (0, 1.0), (0, 1.0)]
[(9, 0.50097954), (0, 0.99999714), (0, 0.9999983), (0, 0.9999978)]
[(9, 0.9195559), (0, 1.0), (0, 0.99989146), (0, 0.99999577)]
[(9, 0.89636683), (0, 1.0), (0, 0.99999803), (0, 1.0)]
[(9, 0.9718955), (0, 1.0), (0, 1.0), (0, 0.9999949)]
[(9, 0.8775721), (0, 0.9999999), (0, 0.9999898), (0, 1.0)]
[(9, 0.8401699), (0, 1.0), (0, 1.0), (0, 0.99999994)]
[(9, 0.9734713), (0, 0.99999994), (0, 0.99999815), (0, 0.9999556)]
[(9, 0.7078

In [14]:
t1 = np.array([9] * 25)
t2 = np.array([9] * 25)
t3 = np.array([9] * 25)
t4 = np.array([9] * 25)
true = np.load('four_subset_FGSM/precise_large_true.csv.npy')
t1 = true
#t2 = true
t3 = true
t4 = true

subset = '2'
res = []
num_img = 25

for x in range(0, 25):
    print(x)
    img = np.load('inputs_0.csv.npy')[x:x+1]
    adv = FGSM(img, subset, t1[x], t2[x], t3[x], t4[x])
    res.append(adv)

res = np.array(res)
np.save('four_subset_FGSM/adv_large_' + subset + '.csv', res)
print_results(num_img, res, img, subset)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
[(0, 0.99999774), (9, 0.37901044), (0, 0.99999934), (0, 0.99990684)]
[(0, 1.0), (9, 0.7397989), (0, 1.0), (0, 0.9999752)]
[(0, 1.0), (9, 0.6102541), (0, 1.0), (0, 1.0)]
[(0, 0.999959), (9, 0.81198215), (0, 0.99999964), (0, 0.9999982)]
[(0, 1.0), (9, 0.6756746), (0, 1.0), (0, 1.0)]
[(0, 0.9999999), (9, 0.9039521), (0, 0.9999995), (0, 0.99998885)]
[(0, 0.9999966), (9, 0.98932165), (0, 1.0), (0, 0.9999974)]
[(0, 0.99999917), (9, 0.72806334), (0, 0.9999966), (0, 0.99997395)]
[(0, 0.9999996), (9, 0.6848696), (0, 1.0), (0, 0.9999983)]
[(0, 0.9999994), (9, 0.89216065), (0, 1.0), (0, 0.99997854)]
[(0, 1.0), (9, 0.67255145), (0, 1.0), (0, 0.99999845)]
[(0, 0.9999953), (9, 0.85964495), (0, 0.99995357), (0, 0.9661808)]
[(0, 0.9999998), (9, 0.70305216), (0, 0.9999997), (0, 0.99999666)]
[(0, 0.99999976), (9, 0.7632202), (0, 0.9999998), (0, 0.99999917)]
[(0, 1.0), (9, 0.5192338), (0, 1.0), (0, 0.9999995)]
[(0, 1.0), (9, 0.40239537), (0

In [15]:
t1 = np.array([9] * 25)
t2 = np.array([9] * 25)
t3 = np.array([9] * 25)
t4 = np.array([9] * 25)
true = np.load('four_subset_FGSM/precise_large_true.csv.npy')
#t1 = true
#t2 = true
t3 = true
t4 = true

subset = '12'
res = []
num_img = 25

for x in range(0, 25):
    print(x)
    img = np.load('inputs_0.csv.npy')[x:x+1]
    adv = FGSM(img, subset, t1[x], t2[x], t3[x], t4[x])
    res.append(adv)

res = np.array(res)
np.save('four_subset_FGSM/adv_large_' + subset + '.csv', res)
print_results(num_img, res, img, subset)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
[(9, 0.64144427), (9, 0.68428504), (0, 1.0), (0, 0.99999857)]
[(9, 0.54069847), (9, 0.9903621), (0, 0.9999964), (0, 0.9999889)]
[(9, 0.89779884), (9, 0.9786326), (0, 0.99999994), (0, 0.9999788)]
[(9, 0.91594833), (9, 0.97199714), (0, 0.99999833), (0, 0.9999946)]
[(9, 0.8338631), (9, 0.96907955), (0, 0.99997807), (0, 0.99356246)]
[(9, 0.991523), (9, 0.99668705), (0, 0.89754015), (0, 0.9976783)]
[(9, 0.5027412), (9, 0.6967536), (0, 0.9999977), (0, 0.9999997)]
[(9, 0.9983365), (9, 0.98506767), (0, 1.0), (0, 0.99999946)]
[(9, 0.58807385), (9, 0.9503155), (0, 0.99999964), (0, 0.99998707)]
[(9, 0.5727107), (9, 0.7419588), (0, 0.9999999), (0, 0.9999999)]
[(9, 0.5074587), (9, 0.5658915), (0, 0.9999945), (0, 0.9999723)]
[(9, 0.98820496), (9, 0.8183459), (0, 0.99985427), (0, 0.9996987)]
[(9, 0.5857722), (9, 0.9202859), (0, 0.9999239), (0, 0.9999693)]
[(9, 0.7499828), (9, 0.8872152), (0, 0.99998134), (0, 0.99999577)]
[(9, 0.66617966

In [16]:
t1 = np.array([9] * 25)
t2 = np.array([9] * 25)
t3 = np.array([9] * 25)
t4 = np.array([9] * 25)
true = np.load('four_subset_FGSM/precise_large_true.csv.npy')
#t1 = true
#t2 = true
#t3 = true
t4 = true

subset = '123'
res = []
num_img = 25

for x in range(0, 25):
    print(x)
    img = np.load('inputs_0.csv.npy')[x:x+1]
    adv = FGSM(img, subset, t1[x], t2[x], t3[x], t4[x])
    res.append(adv)

res = np.array(res)
np.save('four_subset_FGSM/adv_large_' + subset + '.csv', res)
print_results(num_img, res, img, subset)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
[(9, 1.0), (9, 1.0), (9, 1.0), (0, 0.5044012)]
[(9, 0.9999883), (9, 0.9999955), (9, 0.99999976), (0, 0.5618844)]
[(9, 0.9999969), (9, 0.9983369), (9, 0.9999356), (0, 0.9142565)]
[(9, 0.9999998), (9, 0.99999994), (9, 1.0), (0, 0.83353764)]
[(9, 0.99999505), (9, 0.99990726), (9, 0.9999998), (0, 0.58166337)]
[(9, 0.9999239), (9, 0.9988852), (9, 0.999926), (0, 0.9922251)]
[(9, 0.99999994), (9, 0.9999394), (9, 0.9999998), (0, 0.9305545)]
[(9, 0.9999993), (9, 0.99999905), (9, 1.0), (0, 0.939747)]
[(9, 0.99998325), (9, 0.99985117), (9, 0.9999986), (0, 0.8268313)]
[(9, 0.9999869), (9, 0.9999911), (9, 1.0), (0, 0.85674757)]
[(9, 0.9999982), (9, 0.9999991), (9, 0.9999989), (0, 0.8439588)]
[(9, 0.9999999), (9, 0.9999999), (9, 0.99999964), (0, 0.58316034)]
[(9, 0.9999947), (9, 0.9992625), (9, 0.999979), (0, 0.9647589)]
[(9, 0.99999666), (9, 0.99998397), (9, 0.99999946), (0, 0.60818094)]
[(9, 0.91354567), (9, 0.8977523), (9, 0.9912855

In [13]:
t1 = np.array([9] * 25)
t2 = np.array([9] * 25)
t3 = np.array([9] * 25)
t4 = np.array([9] * 25)
true = np.load('four_subset_FGSM/precise_large_true.csv.npy')
#t1 = true
#t2 = true
t3 = true
#t4 = true

subset = '124'
res = []
num_img = 25

for x in range(0, 25):
    print(x)
    img = np.load('inputs_0.csv.npy')[x:x+1]
    adv = FGSM(img, subset, t1[x], t2[x], t3[x], t4[x])
    res.append(adv)

res = np.array(res)
np.save('four_subset_FGSM/adv_large_' + subset + '.csv', res)
print_results(num_img, res, img, subset)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
[(9, 0.9999998), (9, 1.0), (0, 0.7376486), (9, 1.0)]
[(9, 0.9999987), (9, 0.9999936), (0, 0.882104), (9, 0.99999964)]
[(9, 0.9999997), (9, 0.99999934), (0, 0.74172497), (9, 1.0)]
[(9, 1.0), (9, 1.0), (0, 0.7414783), (9, 1.0)]
[(9, 0.99999344), (9, 0.99212754), (0, 0.9985834), (9, 0.999999)]
[(9, 0.9996142), (9, 0.9997664), (0, 0.55646455), (9, 0.99997294)]
[(9, 0.99999994), (9, 0.99999946), (0, 0.83095026), (9, 1.0)]
[(9, 0.99999803), (9, 0.9986726), (0, 0.6143674), (9, 0.9999993)]
[(9, 0.9999999), (9, 0.99993277), (0, 0.89960164), (9, 1.0)]
[(9, 0.89712834), (9, 0.9819264), (0, 0.942775), (9, 0.99099386)]
[(9, 0.9998972), (9, 0.99997956), (0, 0.958597), (9, 0.99999225)]
[(9, 0.99996996), (9, 0.9999122), (0, 0.56907946), (9, 0.9999983)]
[(9, 0.9998381), (9, 0.99998534), (0, 0.9064594), (9, 1.0)]
[(9, 0.9998513), (9, 0.9354496), (0, 0.9913704), (9, 0.99855006)]
[(9, 0.9999905), (9, 0.9998335), (0, 0.9619852), (9, 0.9999995

In [14]:
t1 = np.array([9] * 25)
t2 = np.array([9] * 25)
t3 = np.array([9] * 25)
t4 = np.array([9] * 25)
true = np.load('four_subset_FGSM/precise_large_true.csv.npy')
#t1 = true
#t2 = true
#t3 = true
#t4 = true

subset = '1234'
res = []
num_img = 25

for x in range(0, 25):
    print(x)
    img = np.load('inputs_0.csv.npy')[x:x+1]
    adv = FGSM(img, subset, t1[x], t2[x], t3[x], t4[x])
    res.append(adv)

res = np.array(res)
np.save('four_subset_FGSM/adv_large_' + subset + '.csv', res)
print_results(num_img, res, img, subset)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
[(9, 0.7398214), (9, 0.97781533), (9, 0.99971235), (9, 0.9963289)]
[(9, 0.9999562), (9, 0.9988812), (9, 0.9999856), (9, 0.99999946)]
[(9, 0.70990986), (9, 0.99947286), (9, 0.9999906), (9, 0.99999934)]
[(9, 0.99724996), (9, 0.85146457), (9, 0.948887), (9, 0.99625754)]
[(9, 0.9290623), (9, 0.874156), (9, 0.99656576), (9, 0.9293598)]
[(9, 0.99753857), (9, 0.9971281), (9, 0.9997115), (9, 0.9994789)]
[(9, 0.9997922), (9, 0.999688), (9, 0.99999934), (9, 0.9999947)]
[(9, 0.89536995), (9, 0.95040315), (9, 0.9967535), (9, 0.9987073)]
[(9, 0.99729526), (9, 0.9984706), (9, 0.99989045), (9, 0.9999992)]
[(9, 0.99848026), (9, 0.9973533), (9, 0.999111), (9, 0.9991798)]
[(9, 0.98067385), (9, 0.9996858), (9, 0.99998605), (9, 0.99998456)]
[(9, 0.9914968), (9, 0.9935737), (9, 0.999962), (9, 0.99651873)]
[(9, 0.98846006), (9, 0.96080095), (9, 0.9999988), (9, 0.9999248)]
[(9, 0.99679214), (9, 0.9998061), (9, 0.99910074), (9, 0.99999684)]
[(9,

In [27]:
[452.5, 454.1]
[432.6]
[390.6]
[440.4]

[440.4]